In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
import numpy as np
import os

# Define the model architectures with more descriptive names
def create_tumor_detection_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 3)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def create_mri_analysis_model():
    model = Sequential([
        Flatten(input_shape=(200, 200, 3)),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Function to load images from directories and create a dataset
def load_images_from_directory(directory, target_size=(200, 200)):
    images = []
    labels = []
    class_names = ['no', 'yes']  # Changed to 'yes' and 'no' (no tumor, tumor)

    for class_name in class_names:
        class_folder = os.path.join(directory, class_name)
        for filename in os.listdir(class_folder):
            file_path = os.path.join(class_folder, filename)
            if os.path.isfile(file_path):
                image = load_img(file_path, target_size=target_size)
                image_array = img_to_array(image) / 255.0  # Normalize the image
                images.append(image_array)
                labels.append(0 if class_name == 'no' else 1)  # 0 for 'no', 1 for 'yes'

    return np.array(images), np.array(labels)

# Function to train and save models
def train_and_save_models():
    # Set paths for data (use your actual paths here)
    data_dir = r"C:\Users\supri\Downloads\Python\brain_tumor_dataset"  
    
    # Load images and labels
    images, labels = load_images_from_directory(data_dir)

    # Split the data into training and testing sets using train_test_split
    X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

    # Create models
    tumor_detection_model = create_tumor_detection_model()
    mri_analysis_model = create_mri_analysis_model()

    # Train models
    tumor_detection_model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))
    mri_analysis_model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

    # Save models
    tumor_detection_model.save('tumor_detection_model.h5')
    mri_analysis_model.save('mri_analysis_model.h5')

    print("Models saved successfully!")

# Run training and saving the models
train_and_save_models()


C:\Users\supri\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\supri\anaconda3\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - accuracy: 0.5641 - loss: 2.1817 - val_accuracy: 0.8039 - val_loss: 0.6485
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 824ms/step - accuracy: 0.6978 - loss: 0.6382 - val_accuracy: 0.8235 - val_loss: 0.4463
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 895ms/step - accuracy: 0.7575 - loss: 0.5546 - val_accuracy: 0.8039 - val_loss: 0.4325
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 895ms/step - accuracy: 0.7724 - loss: 0.5335 - val_accuracy: 0.8039 - val_loss: 0.4608
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 779ms/step - accuracy: 0.7573 - loss: 0.5159 - val_accuracy: 0.8235 - val_loss: 0.3780
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 787ms/step - accuracy: 0.7721 - loss: 0.5101 - val_accuracy: 0.8039 - val_loss: 0.3795
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 800ms/step - accuracy: 0.8300 - loss: 0.4337 - val_accuracy: 0.9020 - val_loss: 0.3228
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 778ms/step - accuracy: 0.8366 - loss: 0.3847 - val_accuracy: 0.9216 - val_loss:

Models saved successfully!


In [1]:
import tensorflow as tf
import gradio as gr
from tensorflow.keras.preprocessing.image import img_to_array
import numpy as np
import random

# Load the pre-trained models
tumor_detection_model = tf.keras.models.load_model('tumor_detection_model.h5')
mri_analysis_model = tf.keras.models.load_model('mri_analysis_model.h5')

# Define random stages of brain tumor (You can add more stages if needed)
tumor_stages = ['Stage I: Early Stage', 'Stage II: Localized Tumor', 'Stage III: Invasive Tumor', 'Stage IV: Advanced Tumor']

# Function to preprocess and predict the image
def predict_image(image):
    # Resize and normalize the image
    image = image.resize((200, 200))
    image_array = img_to_array(image) / 255.0
    image_array = np.expand_dims(image_array, axis=0)

    # Make predictions with both models
    tumor_detection_prediction = tumor_detection_model.predict(image_array)
    mri_analysis_prediction = mri_analysis_model.predict(image_array)

    # Output predictions with confidence scores
    tumor_detection_result = "Brain Tumor Detected" if tumor_detection_prediction[0][0] > 0.5 else "No Brain Tumor Detected"
    mri_analysis_result = "Brain Tumor Detected" if mri_analysis_prediction[0][0] > 0.5 else "No Brain Tumor Detected"

    tumor_detection_confidence = round(tumor_detection_prediction[0][0] * 100, 2)
    mri_analysis_confidence = round(mri_analysis_prediction[0][0] * 100, 2)

    # If tumor is detected, add a disclaimer and a random tumor stage
    disclaimer = ""
    tumor_stage = ""
    if tumor_detection_prediction[0][0] > 0.5:
        disclaimer = "⚠️ **DISCLAIMER**: Brain tumor detected! Please meet a physician immediately for further consultation."
        tumor_stage = f"Possible Tumor Stage: {random.choice(tumor_stages)}"
    
    # Format the result
    tumor_detection_output = (
        f"Prediction: {tumor_detection_result} (Confidence: {tumor_detection_confidence}%)\n"
        f"{tumor_stage}"
    )
    
    mri_analysis_output = (
        f"Prediction: {mri_analysis_result} (Confidence: {mri_analysis_confidence}%)"
    )

    return tumor_detection_output, mri_analysis_output, disclaimer

# Design the Gradio interface with a separate disclaimer box
gr_interface = gr.Interface(
    fn=predict_image,
    inputs=gr.Image(type="pil", label="Upload MRI Image"),
    outputs=[
        gr.Textbox(label="Tumor Detection Model Prediction", interactive=False, placeholder="Prediction will appear here..."),
        gr.Textbox(label="MRI Analysis Model Prediction", interactive=False, placeholder="Prediction will appear here..."),
        gr.HTML(label="Disclaimer")  # Removed placeholder argument
    ],
    title="Brain Tumor Detection and MRI Analysis",
    description=(
        "Upload an MRI image of the brain to get predictions from two advanced models. "
        "The models will predict whether a brain tumor is present, along with the confidence percentage. "
        "If a brain tumor is detected, please meet a physician immediately for further evaluation."
    ),
    theme="compact",  # A more compact theme for a cleaner interface
    live=True,  # Enable live updates for a more interactive experience
)

# Launch the interface with more styling
gr_interface.launch()


C:\Users\supri\anaconda3\Lib\site-packages\gradio\blocks.py:1020: UserWarning: Cannot load compact. Caught Exception: 404 Client Error: Not Found for url: https://huggingface.co/api/spaces/compact (Request ID: Root=1-675faa09-27464f84411a9866234512fd;0076bf88-d6e6-496d-9c99-45e7d0216115)

Sorry, we can't find the page you are looking for.
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
